## pNN Balanced

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.metrics import precision_recall_curve, roc_curve, average_precision_score, roc_auc_score

from script import utils
from script.utils import free_mem

from script.datasets import Dataset, FairDataset

sns.set()

In [ ]:
utils.set_random_seed(42)

In [ ]:
def sample(df, amount: int, seed):
    amount = int(amount)
    if amount > df.shape[0]:
        x = []
        
        while amount > 0:
            x.append(df.sample(n=min(amount, df.shape[0]), random_state=seed))
            amount -= df.shape[0]
        
        return pd.concat(x, axis=0)
    
    return df.sample(n=amount, random_state=seed)

In [ ]:
def train_val_test_split(dataset: Dataset, valid_size=0.25, test_size=0.2, seed=utils.SEED):
    sig = dataset.signal
    bkg = dataset.background
    
    # test split
    train_sig, test_sig = train_test_split(sig, test_size=test_size, random_state=seed)
    train_bkg, test_bkg = train_test_split(bkg, test_size=test_size, random_state=seed)
    
    # train-valid split
    train_sig, valid_sig = train_test_split(train_sig, test_size=valid_size, random_state=seed)
    train_bkg, valid_bkg = train_test_split(train_bkg, test_size=valid_size, random_state=seed)
    
    return (train_sig, train_bkg), (valid_sig, valid_bkg), (test_sig, test_bkg)


In [ ]:
class BalancedSequence(tf.keras.utils.Sequence):
    def __init__(self, signal: pd.DataFrame, background: pd.DataFrame, batch_size: int, 
                 features: list, delta=50, balance_signal=True, balance_bkg=True, seed=utils.SEED,
                 sample_mass=False):
        """keras.Sequence that balances the signal (each mA has the same number of events), and backgrounds"""
        self.sig = signal
        self.bkg = background
        
        self.mass = sorted(self.sig['mA'].unique())
        self.mass_delta = float(delta)
        self.should_sample_mass = bool(sample_mass)
        
        self.rnd = np.random.RandomState(seed)  # "slow" but enough for pd.DataFrame.sample
        self.gen = utils.get_random_generator(seed)   # "fast" random generator for `np.random.choice`
        
        self.should_balance_sig = bool(balance_signal)
        self.should_balance_bkg = bool(balance_bkg)
        
        self.features = features
        self.half_batch = batch_size // 2
        
        if self.should_balance_sig:
            self.signals = {m: self.sig[self.sig['mA'] == m] for m in self.mass}
            
            self.sig_batch = self.half_batch // len(self.signals.keys())
        else:
            self.sig_batch = self.half_batch
            
        if self.should_balance_bkg:
            self.bkgs = {k: self.bkg[self.bkg['name'] == k] for k in self.bkg['name'].unique()}
            
            self.bkg_batch = self.half_batch // len(self.bkgs.keys())
        else:
            self.bkg_batch = self.half_batch
    
    def __len__(self):
        return self.sig.shape[0] // self.half_batch
    
    def __getitem__(self, idx):
        if self.should_balance_sig:
            df = [sample(sig, amount=self.sig_batch, seed=self.rnd) for sig in self.signals.values()]
        else:
            df = [sample(self.sig, amount=self.half_batch, seed=self.rnd)]
        
        if self.should_balance_bkg:
            df.extend([sample(bkg, amount=self.bkg_batch, seed=self.rnd) for bkg in self.bkgs.values()])
        else:
            df.append(sample(self.bkg, amount=self.half_batch, seed=self.rnd))
        
        df = pd.concat(df, axis=0)
        
        x = df[self.features].values
        m = np.reshape(df['mA'].values, newshape=(-1, 1))
        y = np.reshape(df['type'].values, newshape=(-1, 1))
        
        if self.should_sample_mass:
            # sample mass (from signal's mA) for background events -> uses all background events
            mask = np.squeeze(y == 0.0)
            m[mask] = self.gen.choice(self.mass, size=np.sum(mask), replace=True).reshape((-1, 1))
        else:
            # take mass from corresponding mass interval (m - delta, m + delta)
            idx = np.digitize(df['dimuon_M'], data.unique_signal_mass, right=True)
            idx = np.clip(idx, a_min=0, a_max=len(data.unique_signal_mass) - 1)
        
            m = np.array(data.unique_signal_mass)[idx]
            m = np.reshape(m, newshape=(-1, 1))
        
        return dict(x=x, m=m), y


In [ ]:
def dataset_from_sequence(sequence: tf.keras.utils.Sequence, prefetch=2):
    def gen():    
        for i in range(len(sequence)):
            yield sequence[0]
            
    tf_data = tf.data.Dataset.from_generator(
        gen, 
        output_types=({'x': tf.float32, 'm': tf.float32}, tf.float32))
    
    return tf_data.prefetch(prefetch)

In [ ]:
def get_data(dataset: Dataset, features: list, case: int, train_batch=128, eval_batch=1024, **kwargs):
    # split data
    train, valid, test = train_val_test_split(dataset, **kwargs)
    
    # create sequences
    train_seq = BalancedSequence(signal=train[0], background=train[1], batch_size=train_batch, 
                                 features=features, sample_mass=case == 1)

    valid_seq = BalancedSequence(signal=valid[0], background=valid[1], batch_size=eval_batch,
                                 features=features, balance_signal=False, balance_bkg=False, sample_mass=False)

    test_seq = BalancedSequence(signal=test[0], background=test[1], batch_size=eval_batch, 
                                features=features, balance_signal=False, balance_bkg=False, sample_mass=False)
    
    # create tf.Datasets
    train_ds = dataset_from_sequence(train_seq)
    valid_ds = dataset_from_sequence(valid_seq)
    test_ds = dataset_from_sequence(test_seq)
    
    return train_ds, valid_ds, test_ds

In [ ]:
def cmsplot(model, dataset, mass: int, category: int, signal: str, delta=50, bins=20, 
            size=(12, 10), legend='best', title='pNN output distribution', seed=utils.SEED,
            path='plot', save=None, show=True, ax=None):
    """Plots the output distribution of the model, along it's weighted significance"""
    
    if ax is None:
        fig = plt.figure(figsize=size)
        ax = fig.gca()
    
    # select both signal and background in interval (m-d, m+d)
    sig = dataset.signal[dataset.signal['mA'] == mass]
    sig = sig[(sig['dimuon_M'] >= mass - delta) & (sig['dimuon_M'] < mass + delta)]

    bkg = dataset.background
    bkg = bkg[(bkg['dimuon_M'] >= mass - delta) & (bkg['dimuon_M'] < mass + delta)]
    
    num_sig = sig.shape[0]
    
    # prepare data
    x = pd.concat([sig[dataset.columns['feature']],
                   bkg[dataset.columns['feature']]], axis=0).values

    y = np.reshape(pd.concat([sig['type'], bkg['type']], axis=0).values, newshape=[-1])
    x = dict(x=x, m=np.ones_like(y[:, np.newaxis]) * mass)
    
    # predict data
    out = model.predict(x=x, batch_size=1024, verbose=0)
    out = np.asarray(out)

    y_sig = np.squeeze(out[y == 1.0])
    y_bkg = np.squeeze(out[y == 0.0])

    w_bkg = bkg['weight'].values
    w_sig = np.ones_like(y_sig)
    
    # plot
    names = dataset.names_df.loc[bkg.index.values]
    df = pd.DataFrame({'Output': y_bkg, 'Bkg': np.squeeze(names), 'weight': w_bkg})
    
    # plot histograms
    sns.histplot(data=df, x='Output', hue='Bkg', multiple='stack', edgecolor='.3', linewidth=0.5, bins=bins,
                 weights='weight', ax=ax,
                 palette={'DY': 'green', 'TTbar': 'red', 'ST': 'blue', 'diboson': 'yellow'})
    
    h_bkg, _ = np.histogram(y_bkg, bins=bins, weights=w_bkg)
    h_bkg = np.sum(h_bkg)
    
    h_sig, _ = np.histogram(y_sig, bins=bins)
    h_sig = np.sum(h_sig)
    
    w_sig = np.ones_like(y_sig) * (h_bkg / h_sig)
    
    ax.hist(y_sig, bins=bins, alpha=0.5, label='signal', color='purple', edgecolor='purple', 
            linewidth=2, hatch='//', histtype='step',
            weights=w_sig)
    
    # compute significance
    sig_mask = np.squeeze(y == 1.0)
    bkg_mask = np.squeeze(y == 0.0)

    cuts = np.linspace(0.0, 1.0, num=bins)
    ams = []
    w = np.concatenate([w_sig, w_bkg], axis=0)
    
    bx = ax.twinx()
    
    s, _ = np.histogram(y_sig, bins=bins, weights=w_sig)
    b, _ = np.histogram(y_bkg, bins=bins, weights=w_bkg)
    
    for i in range(s.shape[0]):
        s_i = np.sum(s[i:])
        b_i = np.sum(b[i:])
        
        ams.append(s_i / np.sqrt(s_i + b_i))

    k = np.argmax(ams)
    
    # add stuff to plot
    bx.grid(False)
    bx.plot(cuts, ams, color='g', label='Significance')

    ax.axvline(x=cuts[k], linestyle='--', linewidth=2, color='g',
               label=f'{round(cuts[k], 3)}: {round(ams[k], 3)}')

    bx.set_ylabel(r'Significance: $s/\sqrt{s+b}$')
    
    leg = ax.get_legend()
    ax.legend(loc='upper left')
    ax.add_artist(leg)
    
    ax.set_xlabel('Class Label Probability')
    ax.set_ylabel('Weighted Num. Events')
    
    # title
    str0 = f'Category-{category} (#bins = {bins})'
    str1 = f'@{(int(mass - delta), int(mass + delta))} dimuon_M (bkg)'
    str2 = f'{title} @ {int(mass)}mA (signal {signal}), {str1}'
    str3 = f'# signal = {sig.shape[0]}, # bkg = {bkg.shape[0]}'
    
    ax.set_title(f'{str0}\n{str2}\n{str3}')
    
    if isinstance(save, str):
        path = utils.makedir(path)
        plt.savefig(os.path.join(path, f'{save}.png'), bbox_inches='tight')
    
    if show:
        plt.show()


In [ ]:
def cutplot(model, dataset, category: int, signal: str, delta=50, bins=20, 
            size=(12, 10), legend='best', title='[pNN] Best Cut vs mA', seed=utils.SEED,
            path='plot', save=None, show=True, ax=None):
    """Plots the value of the best cut as the mass (mA) varies"""
    if ax is None:
        fig = plt.figure(figsize=size)
        ax = fig.gca()
    
    signal = dataset.signal
    backgr = dataset.background
    
    cuts = []
    
    # select both signal and background in interval (m-d, m+d)
    for mass in dataset.unique_signal_mass:
        sig = signal[signal['mA'] == mass]
        sig = sig[(sig['dimuon_M'] >= mass - delta) & (sig['dimuon_M'] < mass + delta)]

        bkg = backgr
        bkg = bkg[(bkg['dimuon_M'] >= mass - delta) & (bkg['dimuon_M'] < mass + delta)]

        # prepare data
        x = pd.concat([sig[dataset.columns['feature']],
                       bkg[dataset.columns['feature']]], axis=0).values

        y = np.reshape(pd.concat([sig['type'], bkg['type']], axis=0).values, newshape=[-1])
        x = dict(x=x, m=np.ones_like(y[:, np.newaxis]) * mass)

        # predict data
        out = model.predict(x=x, batch_size=1024, verbose=0)
        out = np.asarray(out)

        y_sig = np.squeeze(out[y == 1.0])
        y_bkg = np.squeeze(out[y == 0.0])
    
        # computing weights
        w_bkg = bkg['weight'].values
        w_sig = np.ones_like(y_sig)

        h_bkg, _ = np.histogram(y_bkg, bins=bins, weights=w_bkg)
        h_bkg = np.sum(h_bkg)

        h_sig, _ = np.histogram(y_sig, bins=bins)
        h_sig = np.sum(h_sig)

        w_sig = np.ones_like(y_sig) * (h_bkg / h_sig)

        # compute significance
        ams = []

        s, _ = np.histogram(y_sig, bins=bins, weights=w_sig)
        b, _ = np.histogram(y_bkg, bins=bins, weights=w_bkg)

        for i in range(s.shape[0]):
            s_i = np.sum(s[i:])
            b_i = np.sum(b[i:])

            ams.append(s_i / np.sqrt(s_i + b_i))

        k = np.argmax(ams)
        cuts.append(np.linspace(0.0, 1.0, num=bins)[k])  # add cut value
    
    # plot
    ax.plot(dataset.unique_signal_mass, cuts, marker='o', label=f'avg {round(np.mean(cuts), 2)}')
    
    ax.set_xlabel('Mass (GeV)')
    ax.set_ylabel('Best Cut')
    
    ax.legend(loc=str(legend))
    
    # title
    str0 = f'Category-{category} [#bins = {bins}]'
    ax.set_title(f'{title} ({str0})')
    
    if isinstance(save, str):
        path = utils.makedir(path)
        plt.savefig(os.path.join(path, f'{save}.png'), bbox_inches='tight')
    
    if show:
        plt.show()


In [ ]:
def performance_plot(model, dataset, mass: int, category: int, signal: str, delta=50, 
                     bins=20, size=(10, 9), legend='best', seed=utils.SEED, path='plot', save=None):
    """Plots the PR and ROC curves"""
    fig, axes = plt.subplots(nrows=1, ncols=2)
    
    fig.set_figwidth(size[0] * 2)
    fig.set_figheight(size[1])
    
    sig = dataset.signal[dataset.signal['mA'] == mass]
    sig = sig[(sig['dimuon_M'] >= mass - delta) & (sig['dimuon_M'] < mass + delta)]
    
    bkg = dataset.background
    bkg = bkg[(bkg['dimuon_M'] >= mass - delta) & (bkg['dimuon_M'] < mass + delta)]
    
    num_sig = sig.shape[0]
    
    # prepare data
    x = pd.concat([sig[dataset.columns['feature']],
                   bkg[dataset.columns['feature']]], axis=0).values

    y = np.reshape(pd.concat([sig['type'], bkg['type']], axis=0).values, newshape=[-1])
    x = dict(x=x, m=np.zeros_like(y[:, np.newaxis]) * mass)
    
    # predict data
    out = model.predict(x=x, batch_size=1024, verbose=0)
    out = np.asarray(out)
    
    y_sig = np.squeeze(out[y == 1.0])
    y_bkg = np.squeeze(out[y == 0.0])
    
    # compute weights
    w_bkg = bkg['weight'].values
    
    h_bkg, _ = np.histogram(y_bkg, bins=bins, weights=w_bkg)
    h_bkg = np.sum(h_bkg)
    
    h_sig, _ = np.histogram(y_sig, bins=bins)
    h_sig = np.sum(h_sig)
    
    w_sig = np.ones_like(y_sig) * (h_bkg / h_sig)
    w = np.concatenate([w_sig, w_bkg], axis=0)
    
    str1 = f'@{(int(mass - delta), int(mass + delta))} dimuon_M (bkg)'
    
    # PR-curve
    from sklearn.metrics import PrecisionRecallDisplay
    
    PrecisionRecallDisplay.from_predictions(y_true=y, y_pred=out, sample_weight=w, ax=axes[0],
                                            name=f'pNN @ {int(mass)}mA (signal {signal}), {str1}')
    axes[0].set_title(f'[pNN] PR Curve @ {int(mass)}mA (category {category})')
    
    # ROC curve
    from sklearn.metrics import RocCurveDisplay
    
    RocCurveDisplay.from_predictions(y_true=y, y_pred=out, sample_weight=w, ax=axes[1],
                                     name=f'pNN @ {int(mass)}mA (signal {signal}), {str1}')
    axes[1].set_title(f'[pNN] ROC Curve @ {int(mass)}mA (category {category})')
    
    fig.tight_layout()
    
    if isinstance(save, str):
        path = utils.makedir(path)
        plt.savefig(os.path.join(path, f'{save}.png'), bbox_inches='tight')
    
    plt.show()


### Category 1

In [ ]:
VAR_CAT1 = ["dimuon_deltar", "dimuon_deltaphi", "dimuon_deltaeta", "met_pt", 
             "deltar_bjet1_dimuon", "deltapt_bjet1_dimuon", "deltaeta_bjet1_dimuon", 
             "bjet_1_pt", "bjet_1_eta", "deltaphi_bjet1_dimuon",
             "ljet_1_pt", "ljet_1_eta", "bjet_n", "ljet_n"]

In [ ]:
data = Dataset()
data.load(signal='data/new/signal_bassociated_cat1.csv', 
          bkg='data/new/background_cat1.csv', feature_columns=VAR_CAT1)

data.ds.loc[data.ds['ljet_1_eta'] == -10, 'ljet_1_eta'] = -3.0

In [ ]:
# TODO: argument `case`!
train, valid, test = get_data(data, features=VAR_CAT1, train_batch=1024)

In [ ]:
model, checkpoint = utils.get_compiled_pnn(data, save='new/pnn-balanced-cat_1')

In [ ]:
model.fit(x=train, epochs=100, validation_data=valid, verbose=2,
          callbacks=[checkpoint, EarlyStopping(patience=40)])

In [ ]:
utils.load_from_checkpoint(model, path='new/pnn-balanced-cat_1')

In [ ]:
_ = model.evaluate(x=test, verbose=2)

In [ ]:
for mass in data.unique_signal_mass:
    cmsplot(model, data, mass=mass, category=1, signal='bbH',
            path='<YOUR-PATH>', save=f'significance_{int(mass)}mA')

In [ ]:
for mass in data.unique_signal_mass:
    performance_plot(model, data, mass=mass, category=1, signal='bbH',
                     path='<YOUT-PATH>', save=f'curves_{int(mass)}mA')

In [ ]:
cutplot(model, data, category=1, signal='bbH', legend='lower right',
        path='<YOUT-PATH>', save='best-cut_vs_mA')

### Category 2